In [1]:
#initialize parametrized variables

experimentName = ''
runID = ''


In [2]:
# Parameters
runID = "9c994f7241304419bc09661b79f2cb08"
experimentName = "dummyabc"


In [3]:
import jupyter_client
jupyter_client.kernelspec.find_kernel_specs()

{'python3': '/home/pedro/anaconda3/envs/mlflow-7ce0d1002c6479899f7ec502c9b84f5ce61312e3/share/jupyter/kernels/python3'}

In [4]:
import pandas as pd
import numpy as np

import mlflow, mlflow.sklearn, mlflow.pyfunc
from mlflow.pyfunc import PythonModel

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [5]:
#sets the MLflow experiment and active run
mlflow.set_experiment(experiment_name=experimentName)
mlflow.start_run(runID)

<ActiveRun: >

In [6]:
#data loading from storage to be handled by project_name
data = pd.read_csv('/home/pedro/Documentos/work/Jupyter_Python/SuperTopicModeling/BERT-service/dataset/smart-intents.csv', header=None, names=['Input','Intent'])
data = data.sample(15)
data

,Input,Intent
590,Wo finde ich die 3D Modelle?,DE_apps_experience
2572,Wo finde ich eine Instruktion zum Laden.,DE_charging_instruction
3342,Lesen Sie gerne B�cher?,DE_chitchat_bot_book
647,Wie kann man smart media connect Dienste verlä...,DE_apps_live_traffic
6,Ich habe mir grad einen smart Cabrio zugelegt ...,DE_accessories_rims_tires
3431,Welches Fahrzeugmodell gef�llt Ihnen am besten?,DE_chitchat_bot_favoritesmart
2576,Wo finde ich eine Anweisung zum Laden?,DE_charging_instruction
225,Wo finde ich die connecting app?,DE_apps_crossconnect
2281,Radstand forfour Informationen,DE_car_characteristics_wheelbase_forfour
1222,Ich möchte alles über Daimler wissen.,DE_brand_companyfacts


In [7]:
class STMPredProba(PythonModel):
    
    def __init__(self, n=0):
        self._n = n
        self._model = None
        
        
    def fit(self, df):
        from vectorizer import vectorize
        
        df['sentence_vector'] = vectorize(df['Input'])
        X = pd.DataFrame(list(df['sentence_vector']), index=df.index)
        y= df['Intent']
        
        #defining pipe model
        pipeline = Pipeline([('scaler',MinMaxScaler()),
                             ('model', LogisticRegression(C=1,
                                                          solver='saga',
                                                          multi_class='multinomial',
                                                          max_iter=10000))]) 
        #constructing accuracy
        try: #initially can not be done when classes are unitary..
            X_train, X_test, y_train, y_test = train_test_split(X,
                                                                y,
                                                                test_size=0.2,
                                                                stratify = label_series) # to be contructed from previous predictions and respective new labels to avoid retraining
            pipeline.fit(X = X_train, y = y_train) #constructing the model to accuracy -> to be cut out: see above
            acc = pipeline.score(X = X_test,y = y_test)
            print(acc)
            mlflow.log_metric('accuracy',acc)
        except:
            pass
        # training the model to be served
        pipeline.fit(X, y)
        #store the MLflow model 
#        mlflow.sklearn.log_model(pipeline,'SupTopModel')
        self._model = pipeline
        return None

    def predict(self, df, n=0):#, pipe_model, n=0):
        from vectorizer import vectorize
        
        df['sentence_vector'] = vectorize(df['Input'])
        vector_df = pd.DataFrame(list(df['sentence_vector']), index=df.index)        
        prob_df = pd.DataFrame(self._model.predict_proba(vector_df), index=vector_df.index, columns=self._model.classes_)
        
        if n ==0:
            return prob_df
        else:
            top_n = np.argpartition(prob_df.transpose(), -self._n, axis=0)[-self._n:].apply(lambda x:
                                                                                  prob_df.transpose().index.values[
                                                                                      x]).transpose()
            top_n.columns = ["{}".format(i+1) for i in range(self._n)]
            prob_matrix = pd.DataFrame()
            for col in top_n.columns:
                prob_matrix[col] = prob_df.apply(lambda row: row[top_n[col][row.name]], axis=1)
            top_prob = pd.merge(top_n, prob_matrix, right_index=True, left_index=True)
            tuple_class_prob = pd.DataFrame()
            for i in range(1, self._n+1): #constructs the tuple table
                runner_x = '_'.join([str(i), 'x'])
                runner_y = '_'.join([str(i), 'y'])
                tuple_class_prob[i] = pd.DataFrame([top_prob[runner_x], top_prob[runner_y]]).transpose().apply(tuple,
                                                                                                           axis=1)
            tuple_class_prob.columns = [f"top{i+1}"+str(i) for i in range(1, self._n+1)]
            for i in range(len(tuple_class_prob)): #sorts the tuple table
                list_to_sort=tuple_class_prob.iloc[i].tolist()
                list_to_sort.sort(key=lambda x: -x[1])
                tuple_class_prob.iloc[i]=list_to_sort
            return tuple_class_prob

    if __name__ == "__main__":
        print('great')


great


In [8]:
MLflow_model = STMPredProba()

MLflow_model.fit(data)

MLflow_model.predict(data)
#mlflow.pyfunc.log_model(MLflow_model,'SupTopModel')

,DE_accessories_rims_tires,DE_apps_carplay,DE_apps_crossconnect,DE_apps_experience,DE_apps_live_traffic,DE_automobile_taxes,DE_brand_advantages,DE_brand_companyfacts,DE_car_characteristics_wheelbase_forfour,DE_charging_instruction,DE_chitchat_bot_book,DE_chitchat_bot_favoritesmart,DE_chitchat_bot_watson
590,0.008883,0.005754,0.023815,0.890247,0.006802,0.004691,0.007032,0.006377,0.003593,0.016841,0.007061,0.011183,0.007720
2572,0.010671,0.005186,0.005687,0.005908,0.002379,0.004057,0.004127,0.008656,0.001962,0.940094,0.003732,0.003470,0.004071
3342,0.005407,0.005525,0.005611,0.007009,0.004057,0.002913,0.012081,0.006601,0.002398,0.010602,0.926266,0.007975,0.003555
647,0.004706,0.004201,0.010477,0.006914,0.914664,0.005880,0.014854,0.005963,0.007498,0.004291,0.004436,0.006207,0.009910
6,0.903284,0.008555,0.005270,0.009168,0.004540,0.005614,0.009654,0.017917,0.004361,0.015195,0.005639,0.006288,0.004516
3431,0.006115,0.004275,0.003724,0.011964,0.005886,0.003658,0.009591,0.011371,0.003160,0.010985,0.007681,0.909320,0.012269
2576,0.004874,0.003695,0.010462,0.010648,0.003025,0.002911,0.006730,0.007476,0.002226,0.928033,0.006716,0.008057,0.005147
225,0.005449,0.005383,0.894018,0.024103,0.009610,0.005234,0.007162,0.003336,0.003290,0.017025,0.005796,0.004202,0.015390
2281,0.004328,0.005683,0.002851,0.003160,0.008732,0.009058,0.001628,0.009393,0.943541,0.002787,0.002279,0.002880,0.003682
1222,0.009204,0.008002,0.001681,0.003373,0.002022,0.002001,0.003128,0.946201,0.002432,0.010409,0.003703,0.006260,0.001583
